In [ ]:
# ============================
# 1️⃣ 필수 라이브러리 임포트
# ============================
from langchain_community.vectorstores import FAISS
from langchain_openai import OpenAIEmbeddings, ChatOpenAI
from langchain.docstore.document import Document
from langchain.tools import tool
from langchain.utilities import WikipediaAPIWrapper
from langchain.agents import initialize_agent, Tool
import os
from dotenv import load_dotenv

load_dotenv()

OPENAI_API_KEY = os.getenv("OPENAI_API_KEY")

UPSTAGE_API_KEY = os.getenv("UPSTAGE_API_KEY")

TAVILY_API_KEY = os.getenv("TAVILY_API_KEY")

# ============================
# 2️⃣ 카페 메뉴 데이터 로드 및 Document 생성
# ============================
file_path = "../data/cafe_menu_data.txt"

# 데이터 읽기
with open(file_path, "r", encoding="utf-8") as f:
    lines = [line.strip() for line in f if line.strip()]

# 메뉴 단위로 묶기
menu_data = []
temp = {}
for line in lines:
    if line[0].isdigit():  # 새로운 메뉴 시작
        if temp:
            menu_data.append(temp)
            temp = {}
        temp['title'] = line.split(".")[1].strip()
    elif "가격:" in line:
        temp['price'] = line.replace("• 가격:", "").strip()
    elif "주요 원료:" in line:
        temp['ingredients'] = line.replace("• 주요 원료:", "").strip()
    elif "설명:" in line:
        temp['desc'] = line.replace("• 설명:", "").strip()
# 마지막 메뉴 추가
if temp:
    menu_data.append(temp)

# Document 객체 생성
docs = []
for m in menu_data:
    content = f"메뉴: {m['title']}\n가격: {m['price']}\n재료: {m['ingredients']}\n설명: {m['desc']}"
    docs.append(Document(page_content=content, metadata={"menu": m['title']}))

print(f"총 Document 수: {len(docs)}")  # 10개 나와야 함

# ============================
# 3️⃣ FAISS 벡터 DB 생성 및 저장
# ============================
embeddings = OpenAIEmbeddings(api_key=OPENAI_API_KEY, model="text-embedding-3-small")
db = FAISS.from_documents(docs, embeddings)

os.makedirs("./db", exist_ok=True)
db.save_local("../db/cafe_db")
print("FAISS 벡터 DB 생성 완료!")

# ============================
# 4️⃣ 도구 정의 (@tool)
# ============================
# a) 웹 검색 도구
@tool
def tavily_search_func(query: str) -> str:
    """웹에서 최신 정보를 검색하여 반환합니다."""
    # 실제 Tavily API 호출 시 변경 필요
    return f"[웹 검색 결과] '{query}' 관련 최신 정보 제공"

# b) 위키 요약 도구
wiki = WikipediaAPIWrapper()
@tool
def wiki_summary(topic: str) -> str:
    """위키피디아에서 주제에 대한 요약 정보를 반환합니다."""
    return wiki.run(topic)

# c) 로컬 벡터 DB 검색 도구
@tool
def db_search_cafe_func(query: str) -> str:
    """로컬 카페 메뉴 벡터 DB에서 관련 메뉴 정보를 검색하여 반환합니다."""
    db_local = FAISS.load_local(
        "../db/cafe_db",
        embeddings,
        allow_dangerous_deserialization=True  # <-- 이 옵션 추가
    )
    results = db_local.similarity_search(query, k=3)
    return "\n".join([doc.page_content for doc in results])


# ============================
# 5️⃣ LLM과 도구 바인딩
# ============================
llm = ChatOpenAI(model_name="gpt-3.5-turbo", temperature=0)

tools = [
    Tool.from_function(
        func=tavily_search_func,
        name="tavily_search",
        description="웹에서 최신 정보를 검색하여 반환"
    ),
    Tool.from_function(
        func=wiki_summary,
        name="wiki_summary",
        description="위키피디아에서 주제에 대한 요약 정보를 반환"
    ),
    Tool.from_function(
        func=db_search_cafe_func,
        name="db_search_cafe",
        description="로컬 카페 메뉴 DB에서 관련 메뉴 정보를 검색하여 반환"
    ),
]


agent = initialize_agent(tools, llm, agent="zero-shot-react-description", verbose=True)

# ============================
# 6️⃣ 테스트 질문 처리
# ============================
question = "아메리카노의 가격과 특징은 무엇인가요?"
response = agent.run(question)
print("\n===== AI 답변 =====\n")
print(response)
